# Install & setup

## Installation

```{include} includes/installation.md

```

## Sign up & log in

```{include} signup-login.md

```

## Init an instance

Initialize an instance using `lamin init` on the commmand line and these options:

- `storage`: a default storage location for the instance (e.g. `s3://my-bucket`, `gs://my-bucket`, `./my-data-dir`)
- `name` (optional): a name for the instance (e.g., `my-assets`)
- `db` (optional): a Postgres database connection URL, do not pass for SQLite
- `schema` (optional): comma-separated string of schema modules

### Examples

#### Local storage + SQLite

If you are only interested in tracking artifacts and their transformations, init your local SQLite instance via:

```
lamin init --storage ./mydata
```

Mount the Bionty schema module:

```
lamin init --storage mydata --schema bionty
```

#### S3 + SQLite

```
lamin init --storage s3://<bucket_name> --schema bionty
```

#### GCP + Postgres

```
lamin init --storage gs://<bucket_name> --db postgresql://<user>:<pwd>@<hostname>:<port>/<dbname> --schema bionty
```


## Load an instance

Load your own instance:
```
lamin load <instance_name>
````

Load somebody else's instance:
```
lamin load <account_handle/instance_name>
```

## Access settings

Now, let's look at a specific example:

In [ ]:
!lamin init --storage mydata --schema bionty

Print settings:

In [ ]:
!lamin info

Settings persist in `~/.lamin/` and can also be accessed via {class}`lamindb.setup.settings`.

In [ ]:
import lamindb as ln

In [ ]:
ln.setup.settings.user

In [ ]:
ln.setup.settings.instance

```{note}

- The user who creates an instance is its owner. Ownership can be transferred in the hub.
- Advanced users could also consider the Python setup API: {mod}`lamindb.setup`.

```

## Update default storage

It's easiest to see and update default storage in the Python API using {attr}`~lamindb.core.Settings.storage`:

```python
import lamindb as ln
ln.settings.storage  # set via ln.settings.storage = "s3://other-bucket"
#> s3://default-bucket
```

You can also change it using the CLI via

```
lamin set --storage s3://other-bucket
```

## Close an instance

Loading an instance means loading an environment for managing your datasets.

When loading a new instance, you automatically _close_ the previously loaded old instance.

If you want to close the instance without loading a new instance, use `lamin close`

## Migrate an instance

If you are an admin and you haven't set up automated deployments of migrations, you can use two commands to create and deploy migrations:

- `lamin migrate create`
- `lamin migrate deploy`

Unless you manage a custom plugin schema, you'll _never_ need to create a migration.

You'll receive a logged warning when deploying a migration is advisable.

:::{dropdown} What does this warning look like?

Here is an example:

```
% lamin load testdb
🔶 

Your database is not up to date with your installed Python library.

The database misses the following migrations:
[<Migration lnschema_core.0014_rename_ref_field_featureset_registry>, <Migration lnschema_core.0015_artifact_initial_version_artifact_version>]

Only if you are an admin and manage migrations manually, deploy them to the database:
lamin migrate deploy

Otherwise, downgrade your Python library to match the database!

✅ loaded instance: testuser1/testdb
```
:::

### Create a migration

You need to have the schema package installed locally:
```
git clone https://github.com/my-org/lnschema-custom
cd lnschema-custom
pip install -e .
```

Edit the registries in your schema.

Then, call

```
lamin migrate create
```

to create the migration script.

When you're happy, commit them to your GitHub repo, and ideally make a new release.

To deploy the migration call `lamin migrate deploy`.

```{note}

The `lamin` migration commands are a wrapper around Django's migration manager.

```

## Delete an instance

This works as follows. It won't delete your data, just the metadata managed by LaminDB:

In [ ]:
!lamin delete --force mydata